<a href="https://colab.research.google.com/github/svtuck/rankings/blob/master/formats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [73]:
from random import gauss

# Determined Emperically for 2018 mens college
RATING_MU = 863
RATING_SIGMA = 481
PERFORMANCE_VAR_MU = 208
PERFORMANCE_VAR_SIGMA = 52

#Assume each team has some true rating and true variability. Generate performance as a sample from this rating and variability
#Generate teams as a normal normal sample as determined emprically by 2018 results
class Team:
  @staticmethod
  def generate():
    return Team(gauss(RATING_MU, RATING_SIGMA), gauss(PERFORMANCE_VAR_MU, PERFORMANCE_VAR_SIGMA))
  
  def __init__(self, rating, var):
    self.rating = rating
    self.var = var
    
  # We will decide who wins by comparing performance score
  def perform(self):
    return gauss(self.rating, self.var)
  
  def __str__(self):
    return "RATING[%s] VAR[%s]" % (self.rating, self.var)
  
  def __lt__(self, other):
    return str(self) < str(other)
  

for i in range(10):
  print(Team.generate())
  
  

RATING[1035.3491115037352] VAR[176.813604383049]
RATING[1188.2283417264052] VAR[256.34699534759386]
RATING[1658.4050084033197] VAR[211.58404739222632]
RATING[594.3982652896084] VAR[325.6395465744664]
RATING[1014.6888502090683] VAR[231.12657352782233]
RATING[737.2002919300855] VAR[212.27344645671772]
RATING[1137.0781291279332] VAR[220.5781987990763]
RATING[200.88625191647873] VAR[146.42132960553303]
RATING[1423.4786984422742] VAR[189.6785089557069]
RATING[1205.5628173529944] VAR[204.73059596263764]


In [74]:
def naive_tournament(teams):
  return [x[1] for x in sorted([(team.perform(), team) for team in teams], reverse=True)]


section = [Team.generate() for i in range(10)]
results = naive_tournament(section)
for x in results:
  print(x)

RATING[1650.2944654955816] VAR[297.59465129706416]
RATING[1370.234053332821] VAR[221.62818735172823]
RATING[901.7255709038797] VAR[243.69152133535408]
RATING[1332.2514164296224] VAR[219.63042617755835]
RATING[1144.4547387387722] VAR[171.0892163120566]
RATING[1026.978702513904] VAR[141.83681698065828]
RATING[949.6483045165536] VAR[137.20071825865347]
RATING[610.8157929111718] VAR[97.53910941024682]
RATING[334.39985694246604] VAR[267.2064959890181]
RATING[225.70289318556308] VAR[167.1534146002695]


In [75]:
from itertools import combinations
from collections import defaultdict

#Ranking procedure with tiebreaks
def rank_performance(teams, *score_funs):
  if len(teams) == 1:
    return [teams[0][1]]
  score_fun = score_funs[0]
  teams_in_buffer = [x[1] for x in teams]
  scores = sorted([(sum([score_fun(t, rating) for t, rating in wins if t in teams_in_buffer]), wins, team) for wins, team in teams], key=lambda x: x[0], reverse=True)
  ranking = []
  buffer = []
  for (score, wins, team) in scores:
    if not buffer or buffer[0][0] == score:
      buffer.append((wins, team))
    else:
      ranking.extend(rank_performance(buffer, score_funs[1:]))
      buffer = [(wins, team)]
  ranking.extend( rank_performance(buffer))
                  
  return ranking

#Plays a round robin
def pool_play(teams):
  matchups = combinations(teams, 2)
  records = {team: [] for team in teams}
  
  for a, b in matchups:
    a_performance = a.perform()
    b_performance = b.perform()
    if a_performance > b_performance:
      records[a].append((b, a_performance - b_performance ))
    else:
      records[b].append((a, b_performance - a_performance))
   
  num_wins =  [(wins, team) for team, wins in records.items()]
  #Ranking procedure, is 1. number of wins among all teams, then number of wins among tied teams, then performance rating among tied teams (proxy for point diff)
  return rank_performance(num_wins, lambda t, rating: 1, lambda t, rating: 1, lambda t, rating: rating)         
  

results = pool_play(section)
print(len(section))
for x in results:
  print("Team", x)

10
Team RATING[1650.2944654955816] VAR[297.59465129706416]
Team RATING[1332.2514164296224] VAR[219.63042617755835]
Team RATING[901.7255709038797] VAR[243.69152133535408]
Team RATING[1370.234053332821] VAR[221.62818735172823]
Team RATING[949.6483045165536] VAR[137.20071825865347]
Team RATING[1144.4547387387722] VAR[171.0892163120566]
Team RATING[1026.978702513904] VAR[141.83681698065828]
Team RATING[610.8157929111718] VAR[97.53910941024682]
Team RATING[225.70289318556308] VAR[167.1534146002695]
Team RATING[334.39985694246604] VAR[267.2064959890181]


In [76]:
import operator as op
from functools import reduce

def ncr(n, r):
    r = min(r, n-r)
    numer = reduce(op.mul, range(n, n-r, -1), 1)
    denom = reduce(op.mul, range(1, r+1), 1)
    return numer / denom

#Given a set of teams, run a procedure n times and measure controversy
def evaluate(teams, procedure, loss_fn, n=100):
  tally = 0
  for x in range(n):
    result = procedure(teams)
    score = loss_fn(result)
    tally += score
  return tally / n


#If the top n teams as determined by the procedure match the top n teams in reality
#Add 1, else add 0
def top_n_no_order(n=1):
  def select_top(teams):
    actual = sorted(teams, key=lambda x: x.rating, reverse = True)
    if set(actual[:n]) == set(teams[:n]):
      return 1 
    else:
      return 0
  return select_top

#If the top n teams as determined by the procedure match the top n teams in reality
# In order, add 1, else add 0
def top_n_in_order(n=1):
  def select_top(teams):
    actual = sorted(teams, key=lambda x: x.rating, reverse = True)
    if actual[:n] == teams[:n]:
      return 1
    else:
      return 0
  return select_top

#Find expected controversy for k teams
def evaluate_k(*procedures, loss_fn, n=100, k=10):
  tally = [0]*len(procedures)
  for i in range(n):
    teams = [Team.generate() for i in range(k)]
    for j, procedure in enumerate(procedures):
      tally[j] += evaluate(teams, procedure, loss_fn, n=1)
  return [x/n for x in tally]


  
top_n_no_order(n=3)(sorted(section, key=lambda x: x.rating, reverse = True))

1

In [77]:

evaluate_k(pool_play, loss_fn=top_n_no_order(3), n=1000, k=12)

[0.507]

In [0]:
import random
from itertools import zip_longest
def wins_against(a,b,seeds):
  
  return seeds[a-1].perform() > seeds[b-1].perform()

#Play a bracket, returns a new order on the seeds included in the bracket
def play_round(r, seeds):
  if type(r) == int:
    return [r]
  if len(r) == 1:
    return r
  if len(r) == 2:
      a = play_round(r[0], seeds)
      b = play_round(r[1], seeds)
      a_winner = a[0]
      b_winner = b[0]
      if wins_against(a_winner, b_winner, seeds):
        return [a[0], b[0]] + [val for pair in zip_longest(a[1:], b[1:]) for val in pair if val]
      else:
        return [b[0], a[0]] + [val for pair in zip_longest(a[1:], b[1:]) for val in pair if val]
    


#Play a series of brackets
#Plays a bracket, then reseeds and continues (to play for first, second, third etc...)
def bracket(b):
  def play_bracket(seeds):
    results = []
    for r in b:
      ro = play_round(r, seeds)
      results.append(seeds[ro[0] - 1 ])
      seeds = results + [seeds[x-1] for x in ro[1:]] + seeds[len(results) + len(ro[1:]):]
    return seeds
  return play_bracket

#Plays pool play with snake seeding
def pools(n):
  def rv(seeds):
    p = [[] for i in range(n)]
    for i in range(len(seeds)):
      z = i % n
      r = int(i/n) % 2
      if r == 0:
        p[z].append(seeds[i])
      else:
        p[n - z-1].append(seeds[i])
    results = [pool_play(pool) for pool in p]
    result = []
    for i in range(len(seeds)):
      z = i % n
      r = int(i/n) % 2

      if r == 0:
        result.append(results[z][int(i/n)])
      else:
        result.append(results[n - z -1][int(i/n)])
    return result  
  return rv

#Randomly split teams into n pools, and run f on the pools then select k/n from each pool
#Resulting in k teams total. Finally reseed those remaining.
def split(n, f, k):
  def rv(teams):
    random.shuffle(teams)
    pools = [f(teams[int(i*len(teams)/n):int((i+1)*len(teams)/n)]) for i in range(n-1)]
    x = sum([pool[:int(k/n)] for pool in pools], [])
    return naive_tournament(x)
  return rv
  
def progress(n):
  def rv(teams):
    return teams[:n]

#Try to make a twelve team region from 3 sections
sectionals = lambda x: split(3, naive_tournament, 12)(x)


eight_1 = [[[[1,8],[4,5]],[[2,7],[3,6]]]]
six_2 = [[1,2], 
         [2,[[3,6],[4,5]]]]
eight_2_1 = [[[1,4],[2,3]],
[2],
[[3,[6,7]],[4,[5,8]]]]

one_advance = lambda x: bracket(eight_1)(pools(2)(x))
two_advance = lambda x: bracket(six_2)(pools(2)(x))      
three_advance = lambda x: bracket(eight_2_1)(pools(2)(x))



In [89]:
print("Higher is less controversial")
print("Evaluating 36 teams 3 bids", evaluate_k(lambda x: three_advance(sectionals(x)), loss_fn=top_n_no_order(3),n=4000,k=36))
print("Evaluating 36 teams 2 bids", evaluate_k(lambda x: two_advance(sectionals(x)), loss_fn=top_n_no_order(2),n=4000,k=36))
print("Evaluating 36 teams 1 bids", evaluate_k(lambda x: one_advance(sectionals(x)), loss_fn=top_n_no_order(1),n=4000,k=36))
four_game_bracket = [[[1,4],[2,3]],
[2, [3,4]],
[3,[4,[[5,8],[6,7]]]]]
three_advance_four_games = lambda x: bracket(four_game_bracket)(pools(2)(x))
print("Evaluating 36 teams 3 bids using four game bracket",  evaluate_k(lambda x: three_advance_four_games(sectionals(x)), loss_fn=top_n_no_order(3),n=4000,k=36))
a,b = evaluate_k(lambda x: three_advance(sectionals(x)), lambda x: three_advance_four_games(sectionals(x)), loss_fn=top_n_no_order(3),n=4000,k=36)
print("Improvement from adding extra games", round((b -a) * 100), "% absolute", round(100*(b/a -1)), " % relative")


 
eight_2_1_played_out = [
   [
      [
         1,
         4
      ],
      [
         2,
         3
      ]
   ],
   [
      2,
      [
         [
            3,
            [
               5,
               8
            ]
         ],
         [
            4,
            [
               6,
               7
            ]
         ]
      ]
   ]
]

      
two_advance_eight_2_1 = lambda x: bracket(eight_2_1_played_out)(pools(2)(x))                        
[nw, nw_2] = evaluate_k(lambda x: one_advance(naive_tournament(x)), lambda x: two_advance_eight_2_1(naive_tournament(x)), loss_fn=top_n_no_order(2),n=4000,k=8)

print("Evaluating 8 teams 2 bids using one advance bracket no sectionals", nw)
print("Evaluating 8 teams 2 bids using two advance bracket no sectionals", nw_2)

print("Improvement from adding extra games", round((nw_2 - nw) * 100), "% absolute", round(100*(nw_2/nw -1)), " % relative")
[nw, nw_2] = evaluate_k(lambda x: one_advance(naive_tournament(x)), lambda x: two_advance_eight_2_1(naive_tournament(x)), loss_fn=top_n_in_order(1),n=4000,k=8)


print("Evaluating 8 teams 1 bids using one advance bracket no sectionals", nw)
print("Evaluating 8 teams 1 bids using two advance bracket no sectionals", nw_2)

Higher is less controversial
Evaluating 36 teams 3 bids [0.43825]
Evaluating 36 teams 2 bids [0.55925]
Evaluating 36 teams 1 bids [0.66]
Evaluating 36 teams 3 bids using four game bracket [0.48975]
Improvement from adding extra games 4 % absolute 9  % relative
Evaluating 8 teams 2 bids using one advance bracket no sectionals 0.57375
Evaluating 8 teams 2 bids using two advance bracket no sectionals 0.6535
Improvement from adding extra games 8 % absolute 14  % relative
Evaluating 8 teams 1 bids using one advance bracket no sectionals 0.71575
Evaluating 8 teams 1 bids using two advance bracket no sectionals 0.72975


In [79]:
def repeated(f, n=100):
  def rv(teams):
    d = defaultdict(int)
    for i in range(n):
      results = f(teams)
      for j in range(len(results)):
        d[results[j]]+=j
    return [x[1] for x in sorted([(v,k) for k,v in d.items()])]
  return rv


print("Play pool play 1000 times, ten teams select top 3", evaluate_k(repeated(pool_play, n=1000), loss_fn=top_n_no_order(1),n=100,k=10))


Play pool play 1000 times, ten teams select top 3 [0.93]
